## Before you start :
   - These exercises are related to the Exploratory data analysis using matplotlib and seaborn.
   - The datasets for Challenge 2 and 3 are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

# Challenge 1

Answer all questions by drawing a graph. Then explain.

In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

**0. Load seaborn dataset "tips"**

In [118]:
df_tips = pd.DataFrame(sns.load_dataset("tips"))
df_tips.head(10)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


**1. What is the prefered day for eating?**

In [113]:
best_day = pd.DataFrame(df_tips['day'].value_counts())
best_day[best_day["day"] == max(df_tips['day'].value_counts())]



,day
Sat,87


**2. What days of the week has more men than women payers?**

In [211]:
quedan = ['day','sex','size']
df_sex = df_tips[quedan]
dias = df_sex['day'].unique()
print(dias)
#df_sex = df_sex.groupby(['day','sex'], as_index=True)
df_sex.set_index(['day','sex'], inplace=True)

for i in dias:
    num_chicas = df_sex.loc[i,'Female'].sum()
    num_chicos = df_sex.loc[i,'Male'].sum()
    print(i,num_chicas<num_chicos)

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']
Sun size    True
dtype: bool
Sat size    True
dtype: bool
Thur size    False
dtype: bool
Fri size    True
dtype: bool


**3. Do you think bill amounts change considerably depending on smoking?**

**4. Is tip correlated to total_bill? Draw a scatter plot**

**BONUS**: Draw a graph of your choice with 2 numerical variables and 2 categorical variable (HINT: use `hue` and `style`). Do you find it useful?